<h1>Laad de data in</h>

In [1]:
from datasets import load_dataset
import pandas as pd
import math

languages     = ['es', 'it', 'de', 'nl', 'fr', 'en']
querry        = [
                    "Hola cómo estás",  # Spanish
                    "Ciao come stai",  # Italian
                    "Hallo wie geht es dir",  # German
                    "Hoi hoe gaat het",  # Dutch
                    "Bonjour comment ça va",  # French
                    "Hello how are you",  # English
                ]

dataset       = load_dataset("papluca/language-identification")
df_train_temp = pd.DataFrame(dataset['train'])
df_test_temp  = pd.DataFrame(dataset['test'])
df_val_temp   = pd.DataFrame(dataset['validation'])

df_train = df_train_temp[df_train_temp['labels'].isin(languages)]
df_test = df_test_temp[df_test_temp['labels'].isin(languages)]
df_val = df_val_temp[df_val_temp['labels'].isin(languages)]


#df_train['text']

c:\Users\lars\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h1>clean de data</h>
haalt alle speciale characters weg behalve spaties

In [2]:
def drop_special_characters(df):
    special_chars = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
    df_clean = df.copy()
    df_clean['text'] = df_clean['text'].apply(lambda x: ''.join([c for c in x if c not in special_chars]))
    return df_clean

df_train = drop_special_characters(df_train)
df_test = drop_special_characters(df_test)
df_val = drop_special_characters(df_val)

df_val

,labels,text
0,nl,Ik ken geen druk zei Mr Feith de ondersecretar...
1,nl,Hier is mijn advies op basis van mijn persoonl...
2,es,Por el precio este reloj es inmejorable Estilo...
3,it,Il Giappone tace in occasione della tragedia d...
7,nl,Zijn dochter Nina Axelrod vertelde The Associa...
...,...,...
9978,en,The game is fine but they sent the one that wa...
9987,de,Sehr schöner preiswerter Controller Funktionie...
9990,es,El botón de home no funciona
9993,de,Die Textur ist sehr flüssig was an sich kein P...


<h1>Het maken van de N-grammen</h>

In [3]:
def create_ngram_frequency_dict(df, n):
    ngram_freq_dict = {}
    
    for index, row in df.iterrows():
        language = row['labels']
        text = row['text']
        ngram_dict = {}
        for i in range(len(text)-(n-1)):
            ngram = text[i:i+n]
            if ngram in ngram_dict:
                ngram_dict[ngram] += 1
            else:
                ngram_dict[ngram] = 1
         
        if language in ngram_freq_dict:
            ngram_freq_dict[language].update(ngram_dict)
        else:
            ngram_freq_dict[language] = ngram_dict
    
    return ngram_freq_dict

trigram_freq_dict_train = create_ngram_frequency_dict(df_train, 3)
bigram_freq_dict_train = create_ngram_frequency_dict(df_train, 2)
unigram_freq_dict_train = create_ngram_frequency_dict(df_train, 1)
#trigram_freq_dict_train['nl']



<h1>Ophalen totalen nummer frequenties</h>

In [4]:
def count_ngrams(ngram_freq_dict):
    total_ngrams = 0
    ngrams_per_language = {}

    for language, ngram_freq in ngram_freq_dict.items():
        ngrams_count = 0

        for ngram, count in ngram_freq.items():
            ngrams_count += count
        
        total_ngrams += ngrams_count
        ngrams_per_language[language] = ngrams_count

    return total_ngrams, ngrams_per_language


total_trigrams, trigrams_per_language = count_ngrams(trigram_freq_dict_train) # total_trigrams for testing purposes
total_bigrams, bigrams_per_language = count_ngrams(bigram_freq_dict_train) # total_bigrams for testing purposes
total_unigrams, unigrams_per_language = count_ngrams(unigram_freq_dict_train) # total_unigrams for testing purposes

#trigrams_per_language


<h1>Kans per ngram berekenen</h>

In [5]:
def calculate_ngram_chances(ngram_freq_dict, ngrams_per_language):
    ngram_chances = {}

    for language, ngram_freq in ngram_freq_dict.items():
        chances = {}
        total_ngrams = ngrams_per_language[language]

        for ngram, freq in ngram_freq.items():
            chance = freq / total_ngrams
            chances[ngram] = chance

        ngram_chances[language] = chances

    return ngram_chances

trigram_chances_train = calculate_ngram_chances(trigram_freq_dict_train, trigrams_per_language)
bigram_chances_train = calculate_ngram_chances(bigram_freq_dict_train, bigrams_per_language)
unigram_chances_train = calculate_ngram_chances(unigram_freq_dict_train, unigrams_per_language)

bigram_chances_train['nl']


{'Ie': 0.0006329113924050633,
 'em': 0.0006329113924050633,
 'ma': 0.0012658227848101266,
 'an': 0.0006329113924050633,
 'nd': 0.0006329113924050633,
 'd ': 0.0006329113924050633,
 ' h': 0.0006329113924050633,
 'ho': 0.0006329113924050633,
 'ou': 0.0006329113924050633,
 'ud': 0.0006329113924050633,
 'dt': 0.0006329113924050633,
 't ': 0.0006329113924050633,
 ' e': 0.0006329113924050633,
 'ee': 0.0012658227848101266,
 'en': 0.0012658227848101266,
 'n ': 0.002531645569620253,
 'eg': 0.0012658227848101266,
 'ge': 0.0012658227848101266,
 'el': 0.0006329113924050633,
 'l ': 0.0006329113924050633,
 ' v': 0.0006329113924050633,
 'va': 0.0006329113924050633,
 'as': 0.0006329113924050633,
 'st': 0.0018987341772151898,
 'Ba': 0.0006329113924050633,
 'ar': 0.0006329113924050633,
 'ra': 0.0012658227848101266,
 'ac': 0.0006329113924050633,
 'ck': 0.0006329113924050633,
 'k ': 0.0006329113924050633,
 ' O': 0.0006329113924050633,
 'Ob': 0.0006329113924050633,
 'ba': 0.0006329113924050633,
 'am': 0.00

<h1>Functie voor het classificeren van de kansen per taal gegeven een imput querry en een ngram kansen lijst</h1>

$
𝑃( string | l ) = 𝑃( ngram_1 | l ) ∗ 𝑃( ngram_2 | l ) * ... ∗ 𝑃( ngram_n | l )
$


In [6]:
def classify_language_ngrams(query, trigram_chances):
    n = len(list(trigram_chances[languages[0]].keys())[0])
    print(query)
    trigrams = [query[i:i+n] for i in range(len(query)-(n-1))]
    language_chances = {}
    
    for language, chances in trigram_chances.items():
        language_chances[language] = 1.0
        
        for trigram in trigrams:
            if trigram in chances:
                previvous_chance = language_chances[language]
                language_chances[language] *= chances[trigram]
                #print("N gram has a chance of: " + str(chances[trigram]) + "; Total chance is: " + str(previvous_chance) + "; making formula: " + str(chances[trigram]) + " * " + str(previvous_chance) + " = " + str(language_chances[language]))
            else:
                language_chances[language] *= 0.000001  # Smoothing 
                #print("smoothing. New chance is: " + str(language_chances[language]))
    
    return language_chances


In [7]:
def get_highest_language(results):
    highest_language = max(results, key=results.get)
    return highest_language, results

lang, chance_list = get_highest_language(classify_language_ngrams(querry[5], trigram_chances_train))
print("De taal is: " +  lang)
chance_list

Hello how are you
De taal is: en


{'es': 1.056342346865474e-65,
 'it': 4.407827953480813e-65,
 'de': 3.0985908747405165e-69,
 'nl': 2.395791714618603e-64,
 'fr': 1.21843052928069e-68,
 'en': 5.252761406610812e-60}

<h1>Functie voor het classificeren van de kansen per taal gegeven een imput querry, een ngram kansen lijst en een n-1gram kansen lijst (tussen grammen waarbij n-1)</h1>

$
𝑃( string | l ) = \dfrac{𝑝𝑟𝑜𝑑𝑢𝑐𝑡 𝑣𝑎𝑛 𝑎𝑙𝑙𝑒 𝑡𝑟𝑖𝑔𝑟𝑎𝑚𝑚𝑎𝑛 | l}{𝑝𝑟𝑜𝑑𝑢𝑐𝑡 𝑣𝑎𝑛 𝑎𝑙𝑙𝑒 ′𝑡𝑢𝑠𝑠𝑒𝑛′ 𝑏𝑖𝑔𝑟𝑎𝑚𝑚𝑒𝑛 | l}
$


In [14]:
def classify_language_ngrams2(query, trigram_chances, bigram_chances):
    n = len(list(trigram_chances[languages[0]].keys())[0])

    trigrams = [query[i:i+n] for i in range(len(query)-(n-1))]
    between_bigrams = [query[i:i+(n-1)] for i in range(1, len(query)-(n-2))][ : -1] #remove last element because it is not a between bigram
    #print(trigrams)
    #print(between_bigrams)
    print(query)

    language_ngram_chances = {}
    language_between_ngram_chances = {}
    
    for language, chances in trigram_chances.items():
        language_ngram_chances[language] = 1.0
        
        for trigram in trigrams:
            if trigram in chances:
                previvous_chance = language_ngram_chances[language]
                language_ngram_chances[language] *= chances[trigram]
                if(language_ngram_chances[language] <= 0.0): #smallest possible number in python, otherwise it will be 0 wich will cause devision by zero error
                    language_ngram_chances[language] == 2.2250738585072014e-308
                #print("Trigram has a chance of: " + str(chances[trigram]) + "; Total previous is: " + str(previvous_chance) + "; making formula: " + str(chances[trigram]) + " * " + str(previvous_chance) + " = " + str(language_ngram_chances[language]))
            else:
                language_ngram_chances[language] *= 0.000001  # Smoothing
                if(language_ngram_chances[language] <= 0.0): #smallest possible number in python, otherwise it will be 0 wich will cause devision by zero error
                    language_ngram_chances[language] == 2.2250738585072014e-308 
                #print("smoothing. New chance is: " + str(language_ngram_chances[language]))
    
    for language, chances in bigram_chances.items():
        language_between_ngram_chances[language] = 1.0
        
        for bigram in between_bigrams:
            if bigram in chances:
                previvous_chance = language_between_ngram_chances[language]
                language_between_ngram_chances[language] *= chances[bigram]
                if(language_between_ngram_chances[language] <= 0.0): #smallest possible number in python, otherwise it will be 0 wich will cause devision by zero error
                    language_between_ngram_chances[language] == 2.2250738585072014e-308
                #print("Bigram has a chance of: " + str(chances[bigram]) + "; Total chance is: " + str(previvous_chance) + "; making formula: " + str(chances[bigram]) + " * " + str(previvous_chance) + " = " + str(language_between_ngram_chances[language]))
            else:
                language_between_ngram_chances[language] *= 0.000001 # Smoothing
                if(language_between_ngram_chances[language] <= 0.0): #smallest possible number in python, otherwise it will be 0 wich will cause devision by zero error
                    language_between_ngram_chances[language] == 2.2250738585072014e-308
                #print("smoothing. New chance is: " + str(language_between_ngram_chances[language]))

    result = {}
    print(language_ngram_chances)
    print(language_between_ngram_chances)
    for language, chances in trigram_chances.items():
        result[language] = language_ngram_chances[language] / language_between_ngram_chances[language] # devide product trigram chances by product between bigram chances

    return result

In [9]:
def get_highest_language(results):
    highest_language = max(results, key=results.get)
    return highest_language, results

lang, chance_list = get_highest_language(classify_language_ngrams2(querry[3], trigram_chances_train, bigram_chances_train))
print(chance_list)
print("De taal is: " +  lang)

Hoi hoe gaat het
{'es': 3.4840230605844654e-66, 'it': 1.3162211525626696e-63, 'de': 1.9549009828737924e-65, 'nl': 2.240065253168394e-60, 'fr': 1.1754404499671944e-66, 'en': 1.4390184699215944e-63}
{'es': 3.665017956650307e-42, 'it': 8.220506186013874e-40, 'de': 9.277086808043421e-42, 'nl': 7.84475029730923e-42, 'fr': 1.4366949854628115e-44, 'en': 7.727157855132181e-41}
{'es': 9.506155499900295e-25, 'it': 1.6011436799379207e-24, 'de': 2.1072358417287353e-24, 'nl': 2.855495928196392e-19, 'fr': 8.181558798916128e-23, 'en': 1.8622868807653967e-23}
De taal is: nl


<h1>Het testen van de verschillende berekeningen met verschillende ngrams</h1>

In [10]:
def test_model(df, ngram_chances):
    correct = 0
    total = 0
    for index, row in df.iterrows():
        total += 1
        language = row['labels']
        text = row['text']
        #print(text)
        lang, chance_list = get_highest_language(classify_language_ngrams(text, ngram_chances))
        print(chance_list)
        if lang == language:
            correct += 1
    return correct / total

def test_model_2(df, ngram_chances, ngram2_chances):
    correct = 0
    total = 0
    for index, row in df.iterrows():
        total += 1
        language = row['labels']
        text = row['text']
        #print(text)
        lang, chance_list = get_highest_language(classify_language_ngrams2(text, ngram_chances, ngram2_chances))
        print(chance_list)
        if lang == language:
            correct += 1
    return correct / total

test_model(df_val, trigram_chances_train)

Ik ken geen druk zei Mr Feith de ondersecretaris van defensie voor beleid
{'es': 2.146186069e-314, 'it': 2.0798556144859723e-299, 'de': 1.9914432894126066e-297, 'nl': 7.069150258353827e-283, 'fr': 2.267222061317067e-309, 'en': 9.868195290187086e-301}
Hier is mijn advies op basis van mijn persoonlijke ervaring met het afkolven van moedermelk voor tweelingjongens voor een heel jaar
{'es': 0.0, 'it': 0.0, 'de': 0.0, 'nl': 0.0, 'fr': 0.0, 'en': 0.0}
Por el precio este reloj es inmejorable Estilo clásico y mantiene el tiempo bien Estoy muy satisfecho   
{'es': 0.0, 'it': 0.0, 'de': 0.0, 'nl': 0.0, 'fr': 0.0, 'en': 0.0}
Il Giappone tace in occasione della tragedia del 311
{'es': 9.752229628018478e-212, 'it': 1.887419959613926e-198, 'de': 8.142137637123527e-211, 'nl': 1.6465218476567053e-208, 'fr': 3.5263309724429947e-211, 'en': 1.4348279784225965e-203}
Zijn dochter Nina Axelrod vertelde The Associated Press dat hij in zijn slaap stierf blijkbaar aan hartfalen
{'es': 0.0, 'it': 0.0, 'de': 0.0

0.5263333333333333

In [12]:
accuracy_result_classify_1 = {}
accuracy_result_classify_2 = {}
ngram_list = [trigram_chances_train, bigram_chances_train, unigram_chances_train]

for i in range(len(ngram_list)):
    accuracy_result_classify_1[i] = test_model(df_val, ngram_list[i])

print("klaar test 1#################################################################################################################################################################")

for i in range(len(ngram_list)-1):
    accuracy_result_classify_2[i] = test_model_2(df_val, ngram_list[i], ngram_list[i+1])

print(accuracy_result_classify_1)
print(accuracy_result_classify_2)

Ik ken geen druk zei Mr Feith de ondersecretaris van defensie voor beleid
{'es': 2.146186069e-314, 'it': 2.0798556144859723e-299, 'de': 1.9914432894126066e-297, 'nl': 7.069150258353827e-283, 'fr': 2.267222061317067e-309, 'en': 9.868195290187086e-301}
Hier is mijn advies op basis van mijn persoonlijke ervaring met het afkolven van moedermelk voor tweelingjongens voor een heel jaar
{'es': 0.0, 'it': 0.0, 'de': 0.0, 'nl': 0.0, 'fr': 0.0, 'en': 0.0}
Por el precio este reloj es inmejorable Estilo clásico y mantiene el tiempo bien Estoy muy satisfecho   
{'es': 0.0, 'it': 0.0, 'de': 0.0, 'nl': 0.0, 'fr': 0.0, 'en': 0.0}
Il Giappone tace in occasione della tragedia del 311
{'es': 9.752229628018478e-212, 'it': 1.887419959613926e-198, 'de': 8.142137637123527e-211, 'nl': 1.6465218476567053e-208, 'fr': 3.5263309724429947e-211, 'en': 1.4348279784225965e-203}
Zijn dochter Nina Axelrod vertelde The Associated Press dat hij in zijn slaap stierf blijkbaar aan hartfalen
{'es': 0.0, 'it': 0.0, 'de': 0.0

ZeroDivisionError: float division by zero